In [1]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm # 진행률 프로세스 바 모듈

In [2]:
# Version 1
# 베개 상품 리뷰 contents 크롤링 - 첫 페이지
# 네이버 페이 구매평 문구 삭제 -> (2022-03-24 07:52:24 에 등록된 네이버 페이 구매평) -> 삭제
U_A = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0"}
res = req.get("https://review4.cre.ma/bodyluv.kr/products/reviews?product_code=439&iframe_id=crema-product-reviews-2&widget_style=&app=0&parent_url=https%3A%2F%2Fbodyluv.kr%2Fproduct%2F%25EB%25B0%2594%25EB%2594%2594%25EB%259F%25BD-%25EB%25B2%25A0%25EA%25B0%259C-best-6%25EC%25A2%2585%2F439%2Fcategory%2F65%2Fdisplay%2F1%2F%3Fcrema-product-reviews-2-page%3D1%26crema-product-reviews-2-sort%3D10&nonmember_token=&secure_device_token=V29cae8d90df18de998e974d71afa2e6e6618bfd6e9b0c6fa10d0ef740ad85a7319f945aa3cc7cb60d9644f2c1c84b0843&page=1&sort=10&iframe=1&install_method=hardcoded", headers = U_A)
html = bs(res.text, "lxml")
reviews = html.select("div.review_list_v2__message.js-collapsed-review-content.js-translate-text")

# print(len("(2022-03-24 07:52:24 에 등록된 네이버 페이 구매평)")) # 38

review_lst = []
for i in range(len(reviews)) :
    pil_re = reviews[i].text.strip()
    if "네이버 페이 구매평" in pil_re :
        pil_re = pil_re.replace(pil_re[-38:], "")
    review_lst.append(pil_re)
review_lst

['지나가던 베개 유목민의 후기입니다.꼬북배게부터 에어폼 등등 안써본 베개가 없습니다심지어 와이프는 거북이입니다반신반의하면서 구매해본 바디럽 몽실베개거북목이 심하던 와이프님께서 바디럽 양면 몽실베개를 베자마자 잠들더군요너무 편하다면서..ㅎㅎ저도 요즘 목디스크가 살짝 있어서 의사가 개발했다는 모 베개가 영 불편하여 베개를 바꿨는데요즘 잠에서 깨는게 힘들어서 맨날 지각합니다이 베개 만드신분 저 회사 짤리면 책임지셔야해요아직 안 사신분 계시면 두번..아니 세번 사세요다만, 직장 다니시는분들은 사지마세요 늦잠자서 짤립니다',
 '아이가 태열이 심해서 걱정이었는데 베개 바꾼 후로뷰터 붉은기가 많이 사라졌어요 아이 두상만을 강조한 제품보다 다르게 아이 숙면을 위한 베개라그런지 소근소근 너무 잘자요',
 '제가 지금쓰는 베개는.. 타브랜드 ㅇㅇ베개라는걸 쓰는데요!! 엄청 오래써가지구 그 베개 아니면 목도아프고 잠도 잘안오고 그래서 계속 쓰다가.. 요새는 좀 부드럽고 폭신한 베개를 사용하고 싶어져서.. 바디럽 워낙 이불이랑 매트랑 빈백이랑 다 잘써가지구 .. 베개두 써보려구 구매해봤어요!! 생각보다 너무 넓찍해서 순간 잘못주문했나?? 생각했지만..ㅎㅎ 제가 지금 쓰고 있는 베개도 커서..ㅎㅎ 상관없을것같아서 하나만 일단 뜯어봤네요 ㅎㅎ 30일 써보고 만족하면 그대로 사용할거구요 ㅎㅎ혹시나 불편하면.. 환불처리 해도된다고 하셔서 그렇게하려구요 ㅎㅎ 일단 오늘부터  꿀잠을 잘수있길...ㅎㅎ 커버는 지금 쓰고있는 바디럽 마약이불 색상맞춰서 라벤더로 했는데 넘 예쁘고 좋네요!♡♡',
 '쿨쿨~너무 좋아요불면증이 심해 자다 깨다를 반복했었는데요. 요 베개를 사용하고는 푹 잘 잤네요.바디럽~좋은 제품 만들어 주셔서 넘 감사해요^',
 '남편은 높게 베는 걸 좋아하고 전 남편보다는 낮게 베는 편이라 양면몽실 구입해봤어요. 평소 베개 2단으로 베고 자는 남편이 왜 이렇게 낮아보이냐 하더니 막상 베보더니 편안하고 딱 잡아주는 느낌이라 좋답니다ㅎㅎ 저도 오자마자 베보았는데 잠이 솔솔 들어서 

In [3]:
# Version 2
# 베개 상품 리뷰 contents 크롤링 - 1 ~ 10 페이지
# 네이버 페이 구매평 문구 삭제 -> (2022-03-24 07:52:24 에 등록된 네이버 페이 구매평) -> 삭제
# 추출한 리뷰 csv 파일로 export
U_A = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36 Edg/125.0.0.0"}
result = []
for i in tqdm(range(1, 11)) :
    re_url = f"https://review4.cre.ma/bodyluv.kr/products/reviews?app=0&iframe=1&iframe_id=crema-product-reviews-2&install_method=hardcoded&page={i}&parent_url=https%3A%2F%2Fbodyluv.kr%2Fproduct%2F%25EB%25B0%2594%25EB%2594%2594%25EB%259F%25BD-%25EB%25B2%25A0%25EA%25B0%259C-best-6%25EC%25A2%2585%2F439%2Fcategory%2F65%2Fdisplay%2F1%2F&product_code=439&secure_device_token=V29cae8d90df18de998e974d71afa2e6e6618bfd6e9b0c6fa10d0ef740ad85a7319f945aa3cc7cb60d9644f2c1c84b0843&widget_env=100&widget_style="
    res = req.get(re_url, headers = U_A)
    html = bs(res.text, "lxml")
    reviews = html.select("div.review_list_v2__message.js-collapsed-review-content.js-translate-text")
    
    # print(len("(2022-03-24 07:52:24 에 등록된 네이버 페이 구매평)")) # 38
    
    for i in range(len(reviews)) :
        pil_re = reviews[i].text.strip()
        if "네이버 페이 구매평" in pil_re :
            pil_re = pil_re.replace(pil_re[-38:], "")
        result.append(pil_re)
        
len(result) # 50
result

100%|██████████| 10/10 [00:12<00:00,  1.29s/it]


['지나가던 베개 유목민의 후기입니다.꼬북배게부터 에어폼 등등 안써본 베개가 없습니다심지어 와이프는 거북이입니다반신반의하면서 구매해본 바디럽 몽실베개거북목이 심하던 와이프님께서 바디럽 양면 몽실베개를 베자마자 잠들더군요너무 편하다면서..ㅎㅎ저도 요즘 목디스크가 살짝 있어서 의사가 개발했다는 모 베개가 영 불편하여 베개를 바꿨는데요즘 잠에서 깨는게 힘들어서 맨날 지각합니다이 베개 만드신분 저 회사 짤리면 책임지셔야해요아직 안 사신분 계시면 두번..아니 세번 사세요다만, 직장 다니시는분들은 사지마세요 늦잠자서 짤립니다',
 '아이가 태열이 심해서 걱정이었는데 베개 바꾼 후로뷰터 붉은기가 많이 사라졌어요 아이 두상만을 강조한 제품보다 다르게 아이 숙면을 위한 베개라그런지 소근소근 너무 잘자요',
 '제가 지금쓰는 베개는.. 타브랜드 ㅇㅇ베개라는걸 쓰는데요!! 엄청 오래써가지구 그 베개 아니면 목도아프고 잠도 잘안오고 그래서 계속 쓰다가.. 요새는 좀 부드럽고 폭신한 베개를 사용하고 싶어져서.. 바디럽 워낙 이불이랑 매트랑 빈백이랑 다 잘써가지구 .. 베개두 써보려구 구매해봤어요!! 생각보다 너무 넓찍해서 순간 잘못주문했나?? 생각했지만..ㅎㅎ 제가 지금 쓰고 있는 베개도 커서..ㅎㅎ 상관없을것같아서 하나만 일단 뜯어봤네요 ㅎㅎ 30일 써보고 만족하면 그대로 사용할거구요 ㅎㅎ혹시나 불편하면.. 환불처리 해도된다고 하셔서 그렇게하려구요 ㅎㅎ 일단 오늘부터  꿀잠을 잘수있길...ㅎㅎ 커버는 지금 쓰고있는 바디럽 마약이불 색상맞춰서 라벤더로 했는데 넘 예쁘고 좋네요!♡♡',
 '쿨쿨~너무 좋아요불면증이 심해 자다 깨다를 반복했었는데요. 요 베개를 사용하고는 푹 잘 잤네요.바디럽~좋은 제품 만들어 주셔서 넘 감사해요^',
 '남편은 높게 베는 걸 좋아하고 전 남편보다는 낮게 베는 편이라 양면몽실 구입해봤어요. 평소 베개 2단으로 베고 자는 남편이 왜 이렇게 낮아보이냐 하더니 막상 베보더니 편안하고 딱 잡아주는 느낌이라 좋답니다ㅎㅎ 저도 오자마자 베보았는데 잠이 솔솔 들어서 

In [4]:
review_all = {"리뷰" : result}

pd.set_option("display.max_colwidth", None)

df = pd.DataFrame(result, index = range(1, len(result)+1))
df.index.name = "No"
df.columns = ["리뷰 내용"]
df

,리뷰 내용
No,
1,"지나가던 베개 유목민의 후기입니다.꼬북배게부터 에어폼 등등 안써본 베개가 없습니다심지어 와이프는 거북이입니다반신반의하면서 구매해본 바디럽 몽실베개거북목이 심하던 와이프님께서 바디럽 양면 몽실베개를 베자마자 잠들더군요너무 편하다면서..ㅎㅎ저도 요즘 목디스크가 살짝 있어서 의사가 개발했다는 모 베개가 영 불편하여 베개를 바꿨는데요즘 잠에서 깨는게 힘들어서 맨날 지각합니다이 베개 만드신분 저 회사 짤리면 책임지셔야해요아직 안 사신분 계시면 두번..아니 세번 사세요다만, 직장 다니시는분들은 사지마세요 늦잠자서 짤립니다"
2,아이가 태열이 심해서 걱정이었는데 베개 바꾼 후로뷰터 붉은기가 많이 사라졌어요 아이 두상만을 강조한 제품보다 다르게 아이 숙면을 위한 베개라그런지 소근소근 너무 잘자요
3,제가 지금쓰는 베개는.. 타브랜드 ㅇㅇ베개라는걸 쓰는데요!! 엄청 오래써가지구 그 베개 아니면 목도아프고 잠도 잘안오고 그래서 계속 쓰다가.. 요새는 좀 부드럽고 폭신한 베개를 사용하고 싶어져서.. 바디럽 워낙 이불이랑 매트랑 빈백이랑 다 잘써가지구 .. 베개두 써보려구 구매해봤어요!! 생각보다 너무 넓찍해서 순간 잘못주문했나?? 생각했지만..ㅎㅎ 제가 지금 쓰고 있는 베개도 커서..ㅎㅎ 상관없을것같아서 하나만 일단 뜯어봤네요 ㅎㅎ 30일 써보고 만족하면 그대로 사용할거구요 ㅎㅎ혹시나 불편하면.. 환불처리 해도된다고 하셔서 그렇게하려구요 ㅎㅎ 일단 오늘부터 꿀잠을 잘수있길...ㅎㅎ 커버는 지금 쓰고있는 바디럽 마약이불 색상맞춰서 라벤더로 했는데 넘 예쁘고 좋네요!♡♡
4,쿨쿨~너무 좋아요불면증이 심해 자다 깨다를 반복했었는데요. 요 베개를 사용하고는 푹 잘 잤네요.바디럽~좋은 제품 만들어 주셔서 넘 감사해요^
5,남편은 높게 베는 걸 좋아하고 전 남편보다는 낮게 베는 편이라 양면몽실 구입해봤어요. 평소 베개 2단으로 베고 자는 남편이 왜 이렇게 낮아보이냐 하더니 막상 베보더니 편안하고 딱 잡아주는 느낌이라 좋답니다ㅎㅎ 저도 오자마자 베보았는데 잠이 솔솔 들어서 놀랐네요. 성공적인 듯! 바스락 이불세트 베개 커버는 좀 크네요;; 전용 커버를 구입하는 것도 좋겠어요.
6,색상이 카메라에 잘 안담기는데 상세페이지 색처럼 따뜻한 색이라 너무 마음에 들어요
7,몽실베개로바꾸고나서 잠을 잘잤어요~목을 잘받쳐주고 어깨 높이보다 너무크지도 너무낮지도 않아서바르게잘때도옆으로 잘때도 잘받쳐줘서 부담이 잘 잘수있어요~베개가 잠잘때 엄청 중요하다고 생각하는데몽실베개로 바꾸기를 잘했다고 생각해요~저는 만족합니다~
8,기존에 쓰던 베개가 불편했는지 항상 뒤척였던 울 애기ㅠ 베개때문인지 몰랐어요 와..근데 바디럽 베개로 바꾸고 나서는 뒤척이지 않고 완전 꿀잠을 잡니다. 세탁하기도 편하고ㅠㅠ 대박! 애기가 너무 편하게 누워서 자요 넘나행복합니다. 덕분에 육퇴도 빨리 했어요!
9,"제일 먼저 쿨쿨베게 색깔이 맘에 들었어요.메쉬 소재라 끈적임 전혀 없고 앞뒤 위아래로 돌려 가며 사용 가능해요.세탁시 망에 접어 넣어서 해야 하고 1~2일 정도 자연 건조 하면 되더라구요. 베게 크키는 성인이 눕기 딱 좋으며,쿨베게 커버와 비교 해 보면 크기는 조금 작은 편.쿨베게 커버 씌워서 쓸 수도 있을 것 같아요.높이는 머리 대고 눌러 편안히 위치 찾으면 되고 안거나 다리 올리기도 좋더라구요.땀 차지 않게 통풍되는 베게라 진짜 맘에 들어요.더 구매하려구요.^^"


In [5]:
df.to_csv("../data/pillow_review.csv", encoding = "utf-8-sig")